In [ ]:
import pandas as pd
import warnings
from unidecode import unidecode
from sportsreference.ncaab.roster import Player
from re import search
from tqdm import tqdm, tqdm_notebook
import numpy as np
warnings.filterwarnings('ignore')

In [ ]:
from sportsreference.ncaab.roster import Player

In [ ]:
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 500)

In [ ]:
df = pd.read_csv("NBA_data.csv")

In [ ]:
#remove accents 
clean_player = [unidecode(player) for player in df["Player"]]
clean_player[:10]

In [ ]:
#make player ids
suffix = "-1"
makes_ids = [player.replace(".", "").replace("'", "").replace(" ", "-").lower()+suffix for player in clean_player]
makes_ids[:10]

In [ ]:
#see which players are generating errors when trying to call college data 
for player in makes_ids:
    try:
        college = Player(player)
    except:
        print(f'Error for {player} in index # {makes_ids.index(player)}')
        


#exceptions
deeandre hullett, cc 23
kedrick brown, cc, 78
ronald murray, cc, 93
qyntel woods, cc, 128
jerome beasely, cc, 139
donta smith, cc, 172
luc-richard-mbah-a-moute-1, 337
bill-walker-1, 343
patrick-mills-1, 398
jeff-pendergraph-1, 403
bj-mullens-1, 424
bernard james, cc, 533
thomas-robinson-2, 555
ricky ledo, cc, 579
roy-devyn-marble-1, 644
kyle-anderson-3, 650
stephen-zimmermanjr-1, 720
kahlil-felder-1, 732
jakob-poeltl-1, 748
taurean-prince-1, 750
edrice-adebayo-1, 799
malik-milton-1, 830
raymond-spalding-1, 831
marvin-bagleyiii-1, 839
lonnie-walker-2, 853

In [ ]:
#make list to fix incorrect player id's 
replace = [('luc-richard-mbah-a-moute-1', 337), ('bill-walker-1', 343), ('patrick-mills-1', 398), ('jeff-pendergraph-1', 403), 
          ('bj-mullens-1', 424),('thomas-robinson-2', 555), ('roy-devyn-marble-1', 644), ('kyle-anderson-3', 650), 
           ('stephen-zimmermanjr-1', 720), ('kahlil-felder-1', 732),('jakob-poeltl-1', 748), ('taurean-prince-1', 750),
           ('edrice-adebayo-1', 799), ('malik-milton-1', 830),('raymond-spalding-1', 831), ('marvin-bagleyiii-1', 839),
           ('lonnie-walker-2', 853)]

In [ ]:
#replace incorrect id's
for player in replace:
    makes_ids[player[1]] = player[0]

In [ ]:
#list players to remove
remove = ['deeandre-hulett-1', 'kedrick-brown-1', 'ronald-murray-1', 'qyntel-woods-1',
'jerome-beasley-1', 'donta-smith-1','bernard-james-1', 'ricky-ledo-1']
no_cc = [player for player in makes_ids if player not in remove] 
len(no_cc)

In [ ]:
#list index of players to remove
remove_nums = [23, 78, 93, 128, 139, 172, 533, 579]

In [ ]:
#remove same index rows from df
clean_df = df.drop(remove_nums)

In [ ]:
#add Player_id column to lean NBA stats df
clean_df['Player_id'] = no_cc

In [ ]:
#get colleges of players to make sure we have the correct players
college_list = []
for player in tqdm_notebook(clean_df["Player_id"]):
    try:
        college = Player(player).team_abbreviation
        college_list.append(college)
    except:
        print(f'Error for {player}')
        

In [ ]:
#replace hyphenated shcool names
college_list = [school.replace("-", " ") for school in college_list]

In [ ]:
#add to clean_df
clean_df['College_id'] = college_list

In [ ]:
#fix college abbreviations
clean_df["College_id"]= (clean_df["College_id"]
.replace("connecticut", "uconn").replace("louisiana state", "lsu").replace("southern california", "usc")
.replace('north carolina', 'unc').replace('southern methodist', 'smu').replace('brigham young', 'byu')
.replace('nevada las vegas', 'unlv').replace('mississippi', 'ole miss').replace('north carolina state', 'nc state')
.replace('massachusetts', 'umass').replace('pittsburgh', 'pitt').replace('louisiana lafayette', 'la-lafayette'))


In [ ]:
#convert College column to lower case 
clean_df['College'] = clean_df['College'].str.lower() 

In [ ]:
#create word column used for college matching
word = [words.split()[0] for words in clean_df['College_id']]
clean_df['word'] = word

In [ ]:
def college_wrong(df):
    global wrong_players
    wrong_palyers = []
    for index, row in df.iterrows():
    if search(row["word"], row["College"]):
        print(row["Player"], "at ", index, " matches")
    else:
        print(row["Player"], "at ", index, " does NOT match")
        wrong_players.append((row["Player"]))
    

In [ ]:
len(wrong_players)

In [ ]:
select = clean_df["Player"].isin(wrong_players)
wrong_df = clean_df[select]

In [ ]:
#create list of players to remove
remove2 = ['ernest-brown-1', 'cory-hightower-1', ]

In [ ]:
#create list of player ids to update
replace = [('ken-johnson-1', 'ken-johnson-2'), ('mike-dunleavy-1', 'mike-dunleavy-2'), ('david-young-1','dave-young-1'),
          ('bobby-jones-1', 'bobby-jones-2'), ('dee-brown-1', 'dee-brown-2'), ('gerald-henderson-1', 'gerald-henderson-2'),
          ('james-johnson-1', 'james-johnson-2'), ('wesley-johnson-1', 'wes-johnson-1'),
          ('james-anderson-1', 'james-anderson-3'), ('derrick-williams-1', 'derrick-williams-2'),
          ('tristan-thompson-1','tristan-thompson-2'), ('jordan-hamilton-1', 'jordan-hamilton-2'),
          ('mike-scott-1', 'mike-scott-4'), ('justin-hamilton-1', 'justin-hamilton-2'), ('anthony-davis-1', 'anthony-davis-5'),
          ('jared-cunningham-1', 'jared-cunningham-2'), ('ray-mccallum-1', 'ray-mccallum-2'), ('glen-rice-1', 'glen-rice-jr-1'),
          ('ryan-kelly-1', 'ryan-kelly-3'), ('reggie-bullock-1', 'reggie-bullock-2'), ('tim-hardaway-1', 'tim-hardaway-jr-1'),
          ('glenn-robinson-1', 'glenn-robinson-2'),('joe-harris-1', 'joe-harris--1'),
          ('marcus-thornton-1', 'marcus-thornton-3'), ('joe-young-1', 'joseph-young-1'),
          ('josh-richardson-1', 'josh-richardson-2'), ('anthony-brown-1', 'anthony-brown-5'), ('aaron-white-1','aaron-white-3'),
          ('devin-booker-1', 'devin-booker-2'), ('larry-nance-1', 'larry-nance-2'),
          ('demetrius-jackson-1', 'demetrius-jackson-2'), ('sterling-brown-1', 'sterling-brown-2'),
          ('frank-jackson-1', 'frank-jackson-2'), ('josh-jackson-1', 'josh-jackson-2'), ('dennis-smith-1', 'dennis-smithjr-1'),
          ('justin-jackson-1', 'justin-jackson-4'), ('justin-jackson-1', 'justin-jackson-5'),
          ('bruce-brown-1', 'bruce-brown-2'), ('gary-trent-1', 'gary-trentjr-1'), ('vince-edwards-1', 'vince-edwards-2'),
          ('troy-brown-1', 'troy-brown-5'), ('jaren-jackson-1', 'jaren-jacksonjr-1'), 
          ('wendell-carter-1', 'wendell-carterjr-1'),('jerome-robinson-1', 'jerome-robinson-2'),
          ('marcus-williams-1', 'marcus-williams-2'), ('michael-porter-1', 'michael-porterjr-1'),
          ('robert-williams-1', 'robert-williams-3')
          ]

In [ ]:
#update player ids in clean_df
for player in replace:
    clean_df.replace(to_replace = player[0],
                    value = player[1], inplace = True)

In [ ]:
#input justin jackson 5
clean_df.loc[812, 'Player_id'] = 'justin-jackson-5'

In [ ]:
#remove players not in NCAA
clean_df = clean_df[~clean_df['Player_id'].isin(remove2)]

In [ ]:
#test player ids one more time
college_list = []
for player in tqdm_notebook(clean_df["Player_id"]):
    try:
        college = Player(player).team_abbreviation
        college_list.append(college)
    except:
        print(f'Error for {player}')
        

In [ ]:
#replace hyphenated shcool names
college_list = [school.replace("-", " ") for school in college_list]

In [ ]:
#add to clean_df
clean_df['College_id'] = college_list

In [333]:
def college_abv(df):
    df["College_id"]= (df["College_id"]
                             .replace("connecticut", "uconn").replace("louisiana state", "lsu")
                             .replace("southern california", "usc")
                             .replace('north carolina', 'unc').replace('southern methodist', 'smu')
                             .replace('brigham young', 'byu').replace('nevada las vegas', 'unlv')
                             .replace('mississippi', 'ole miss').replace('north carolina state', 'nc state')
                             .replace('massachusetts', 'umass').replace('pittsburgh', 'pitt')
                             .replace('louisiana lafayette', 'la-lafayette'))

In [ ]:
#fix college abbreviations
college_abv(clean_df)

In [339]:
#create function to prepare college test
def college_prep(df):
    word = [words.split()[0] for words in df['College_id']]
    df['word'] = word
    return df.head()
    

In [ ]:
college_prep(clean_df)

In [ ]:
#test player ids one more time
college_wrong(clean_df)

In [ ]:
len(wrong_players)

In [ ]:
#all 'wrong players' are fine in this case.
wrong_players

In [ ]:
#re-index clean_df
clean_df.reset_index(drop=True, inplace=True)

In [ ]:
#create  list to hold dictionary data and list for desired ncaa stats
ncaa_data = []

stats = ['player_id','name','assist_percentage', 'assists', 'block_percentage','blocks','effective_field_goal_percentage',
        'field_goal_percentage', 'free_throw_attempt_rate', 'free_throw_percentage', 'minutes_played',
        'points', 'steal_percentage', 'steals', 'three_point_percentage', 'three_pointers', 'total_rebound_percentage',
         'total_rebounds', 'true_shooting_percentage', 'turnover_percentage','turnovers', 'usage_percentage', 'box_plus_minus', 'games_played', 'height', 'win_shares_per_40_minutes']



In [ ]:
#insert player stats into ncaa_data
ncaa_data = []
for player in tqdm_notebook(clean_df["Player_id"]):
    ncaa = Player(player)
    player_list = []
    player_dict = {}
    for stat in stats:
        try:
            var = eval(f'ncaa.{stat}')
            player_list.append((stat, var))
        except:
            var = np.nan
            print(player,stat, var)
            player_list.append((stat, var))
    player_dict.update(player_list)
    ncaa_data.append(player_dict)
    

In [ ]:
#convert ncaa_data to df
ncaa_df = pd.DataFrame(ncaa_data)


In [ ]:
#add college to ncaa_df
ncaa_df["college"] = clean_df["College_id"]
ncaa_df["class"] = clean_df["Class"]

In [ ]:
#create new dataframe which re-orders columns and drops those with a lot of NaN's
ncaa2_df = ncaa_df[["player_id", "name",  "class", "college", "assists", "blocks", "effective_field_goal_percentage",
                    "field_goal_percentage", "free_throw_attempt_rate","free_throw_percentage", "games_played", "height",
                    "minutes_played", "points","steals", "three_point_percentage", "three_pointers", "total_rebounds",
                    "true_shooting_percentage", "turnover_percentage", "turnovers", "win_shares_per_40_minutes" ]]

In [ ]:
ncaa2_df.fillna(ncaa2_df.mean(), inplace=True)

In [ ]:
#stats per 40  list
per_40 = [ "assists", "blocks", "points", "steals", "three_pointers", "total_rebounds", "turnovers"]

In [ ]:
#add suffix for column titles
suff = "_per_40"
for_40 = [stat+suff for stat in per_40]

In [ ]:
#zip two lists together
combined_40 = list(zip(per_40,for_40))

In [ ]:
#check combined list
combined_40

In [ ]:
#run for loop to create per_40 columns
for stat in combined_40:
    ncaa2_df[stat[1]] = (ncaa2_df[stat[0]] /ncaa2_df["minutes_played"])*40

In [ ]:
#fill nan's with column mean
ncaa2_df.fillna(ncaa2_df.mean())

In [ ]:
#convert height to inches

height = [ guy.split('-') for guy in ncaa2_df["height"]]
inches = [(int(dim[0])*12) + int(dim[1]) for dim in height]

In [ ]:
ncaa2_df['height'] = inches

In [ ]:
#export to csv
ncaa2_df.to_csv("NCAA_data.csv",index = False, header = True)

In [ ]:
ncaa2_df.head()

In [ ]:
id_df = ncaa2_df[["player_id", "name"]]
id_df.rename(columns = {'name':'Player'}, inplace = True) 

In [ ]:
id_df.head()

In [ ]:
nba_df = pd.merge(df, id_df, on='Player')

In [ ]:
nba_df.to_csv("NBA_data.csv",index = False, header = True)

In [379]:
#pull in 2020 NBA Draft prospects
df_2020 = pd.read_csv("2020_draft_data/2020_prospects.csv")


In [380]:
# Get rows where year == intl.
intl = df_2020[ df_2020['Year'] == "Intl." ].index

# Delete these row indexes from dataFrame
df_2020.drop(intl, inplace=True)

In [381]:
#re-index df
df_2020.reset_index(drop=True, inplace=True)
df_2020.head()

,Rank,Name,School,Year
0,1,Anthony Edwards,Georgia,Fr.
1,2,James Wiseman,Memphis,Fr.
2,4,Obi Toppin,Dayton,So.
3,5,Jaden McDaniels,Washington,Fr.
4,6,Nico Mannion,Arizona,Fr.


In [382]:
#remove accents 
clean_player_2020 = [unidecode(player) for player in df_2020["Name"]]

In [383]:
#make player ids
suffix = "-1"
ids_2020 = [player.replace(".", "").replace("'", "").replace(" ", "-").lower()+suffix for player in clean_player_2020]

In [299]:
#see which players are generating errors when trying to call college data 
for player in tqdm_notebook(ids_2020):
    try:
        college = Player(player)
    except:
        print(f'Error for {player} in index # {ids_2020.index(player)}')
        

Error for obi-toppin-1 in index # 2
Error for vernon-carey-1 in index # 7
Error for josh-green-1 in index # 11
Error for jay-scrubb-1 in index # 28
Error for nick-richards-1 in index # 31
Error for derrick-alston-jr-1 in index # 42
Error for cam-mack-1 in index # 51


obadiah-toppin-1
vernon-careyjr-1
josh-green-2
remove jay scrubb
nick-richards-2
derrick-alston-2
cameron-mack-1

In [384]:
replace_2020 = [('obadiah-toppin-1', 2), ('vernon-careyjr-1', 7), ('josh-green-2', 11), ('nick-richards-2', 31), 
          ('derrick-alston-2', 42),('cameron-mack-1', 51)]

In [385]:
#replace incorrect id's
for player in replace_2020:
    ids_2020[player[1]] = player[0]

In [386]:
#remove players not in NCAA
df_2020 = df_2020.drop([28])
ids_2020.pop(28)

'jay-scrubb-1'

In [387]:
#reest index
#re-index clean_df
df_2020.reset_index(drop=True, inplace=True)

In [389]:
df_2020['Player_id'] = ids_2020 

In [391]:
#define function to get player_id colleges
def college_try(dataframe):
    global college_list
    college_list = []
    for index, row in tqdm_notebook(dataframe.iterrows()):
        try:
            college =  Player(row["Player_id"]).team_abbreviation
            college_list.append(college)
        except:
            print('Error for', row["Player_id"], 'at index #', index)


In [308]:
college_try(df_2020)

Error for reggie-perry-1 at index # 22


In [392]:
#insert correct value for player #22
ids_2020[22] = 'reggie-perry-2' 

In [393]:
df_2020['Player_id'] = ids_2020 

In [395]:
#run college_try again
college_try(df_2020)

In [396]:
#replace hyphenated shcool names
college_list = [school.replace("-", " ") for school in college_list]

In [397]:
#add to  df
df_2020['College_id'] = college_list

In [398]:
college_abv(df_2020)

In [399]:
college_prep(df_2020)

,Rank,Name,School,Year,Player_id,College_id,word
0,1,Anthony Edwards,Georgia,Fr.,anthony-edwards-1,missouri state,missouri
1,2,James Wiseman,Memphis,Fr.,james-wiseman-1,memphis,memphis
2,4,Obi Toppin,Dayton,So.,obadiah-toppin-1,dayton,dayton
3,5,Jaden McDaniels,Washington,Fr.,jaden-mcdaniels-1,washington,washington
4,6,Nico Mannion,Arizona,Fr.,nico-mannion-1,arizona,arizona


In [400]:
#convert School column to lower case
df_2020['School'] = df_2020['School'].str.lower()

df_2020.head()

,Rank,Name,School,Year,Player_id,College_id,word
0,1,Anthony Edwards,georgia,Fr.,anthony-edwards-1,missouri state,missouri
1,2,James Wiseman,memphis,Fr.,james-wiseman-1,memphis,memphis
2,4,Obi Toppin,dayton,So.,obadiah-toppin-1,dayton,dayton
3,5,Jaden McDaniels,washington,Fr.,jaden-mcdaniels-1,washington,washington
4,6,Nico Mannion,arizona,Fr.,nico-mannion-1,arizona,arizona


In [373]:
wrong_players = []
for index, row in df_2020.iterrows():
    if search(row["word"], row["School"]):
        print(row["Name"], "at ", index, " matches")
    else:
        print(row["Name"], "at ", index, " does NOT match")
        wrong_players.append((row["Name"], index))

Anthony Edwards at  0  does NOT match
James Wiseman at  1  matches
Obi Toppin at  2  matches
Jaden McDaniels at  3  matches
Nico Mannion at  4  matches
Daniel Oturu at  5  matches
Cole Anthony at  6  does NOT match
Vernon Carey at  7  matches
Tyrese Haliburton at  8  matches
Onyeka Okongwu at  9  matches
Isaac Okoro at  10  matches
Josh Green at  11  matches
Zeke Nnaji at  12  matches
Isaiah Stewart at  13  does NOT match
Ashton Hagans at  14  matches
Precious Achiuwa at  15  matches
Aaron Nesmith at  16  matches
Tyrese Maxey at  17  matches
Jordan Nwora at  18  matches
Ochai Agbaji at  19  matches
Paul Reed at  20  does NOT match
Jalen Smith at  21  matches
Reggie Perry at  22  matches
Devon Dotson at  23  matches
Udoka Azubuike at  24  matches
CJ Elleby at  25  matches
Landers Nolley at  26  does NOT match
Devin Vassell at  27  matches
Cassius Stanley at  28  matches
Saddiq Bey at  29  matches
Nick Richards at  30  matches
Elijah Hughes at  31  matches
Tyler Bey at  32  matches
Chris

In [374]:
wrong_players

[('Anthony Edwards', 0),
 ('Cole Anthony', 6),
 ('Isaiah Stewart', 13),
 ('Paul Reed', 20),
 ('Landers Nolley', 26),
 ('Chris Smith', 33),
 ('Malik Fitts', 42),
 ('Desmond Bane', 49)]

### players to replace
anthony-edwards-2
isaiah-stewart-2
paul-reed-5
landers-nolleyii-1
chris-smith-19

In [401]:
update_2020 = [('anthony-edwards-1', 'anthony-edwards-2'), ('isaiah-stewart-1','isaiah-stewart-2'),
               ('paul-reed-1', 'paul-reed-5'),('landers-nolley-1', 'landers-nolleyii-1'),
               ('chris-smith-19', 'chris-smith-19')]

In [402]:
#update player ids 
for player in update_2020:
    df_2020.replace(to_replace = player[0],
                    value = player[1], inplace = True)

In [404]:
#insert player stats into ncaa_data
ncaa_data = []
for player in tqdm_notebook(df_2020["Player_id"]):
    ncaa = Player(player)
    player_list = []
    player_dict = {}
    for stat in stats:
        try:
            var = eval(f'ncaa.{stat}')
            player_list.append((stat, var))
        except:
            var = np.nan
            print(player,stat, var)
            player_list.append((stat, var))
    player_dict.update(player_list)
    ncaa_data.append(player_dict)

In [405]:
#convert ncaa_data to df
draft_df = pd.DataFrame(ncaa_data)


In [406]:
#add college to ncaa_df
draft_df["college"] = df_2020["College_id"]


In [408]:
draft2020_df = draft_df[["player_id", "name", "college", "assists", "blocks", "effective_field_goal_percentage",
                    "field_goal_percentage", "free_throw_attempt_rate","free_throw_percentage", "games_played", "height",
                    "minutes_played", "points","steals", "three_point_percentage", "three_pointers", "total_rebounds",
                    "true_shooting_percentage", "turnover_percentage", "turnovers", "win_shares_per_40_minutes" ]]

In [409]:
draft2020_df.fillna(draft2020_df.mean(), inplace=True)

In [410]:
for stat in combined_40:
    draft2020_df[stat[1]] = (draft2020_df[stat[0]] /draft2020_df["minutes_played"])*40

In [411]:
height = [ guy.split('-') for guy in draft2020_df["height"]]
inches = [(int(dim[0])*12) + int(dim[1]) for dim in height]
draft2020_df['height'] = inches

In [412]:
draft2020_df.to_csv("Draft_data.csv",index = False, header = True)

In [413]:
df = pd.read_csv("NCAA_data.csv")

In [414]:
df.replace(to_replace ="Star", 
                 value ="Player", inplace=True) 
df.replace(to_replace ="Above Avg", 
                 value ="Player", inplace=True)
df.replace(to_replace ="Below Avg", 
                 value ="Player", inplace=True)

In [415]:
df.head(10)

,player_id,name,class,college,assists,blocks,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,games_played,height,minutes_played,points,steals,three_point_percentage,three_pointers,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,win_shares_per_40_minutes,assists_per_40,blocks_per_40,points_per_40,steals_per_40,three_pointers_per_40,total_rebounds_per_40,turnovers_per_40
0,kenyon-martin-1,Kenyon Martin,Player,cincinnati,142,292,0.587,0.586,0.494,0.581,116,81,2042.000000,1279,124,0.222000,2.0,869,0.592,14.300000,139.00000,0.346000,2.781587,5.719882,25.053869,2.428991,0.039177,17.022527,2.722821
1,aj-guyton-1,A.J. Guyton,Player,indiana,403,52,0.541,0.455,0.249,0.790,128,73,3260.000000,2100,129,0.414000,283.0,429,0.572,13.600000,218.00000,0.145000,4.944785,0.638037,25.766871,1.582822,3.472393,5.263804,2.674847
2,jake-voskuhl-1,Jake Voskuhl,Player,uconn,124,193,0.542,0.542,0.555,0.656,138,83,2184.000000,863,86,0.329163,0.0,880,0.573,23.000000,160.00000,0.237000,2.271062,3.534799,15.805861,1.575092,0.000000,16.117216,2.930403
3,khalid-el-amin-1,Khalid El-Amin,Bust,uconn,479,10,0.486,0.416,0.257,0.822,108,70,2149.000000,1650,186,0.355000,195.0,319,0.527,15.600000,188.00000,0.158000,8.915775,0.186133,30.711959,3.462075,3.629595,5.937645,3.499302
4,mike-smith-1,Mike Smith,Bust,louisiana monroe,147,42,0.507,0.428,0.357,0.769,55,80,1938.000000,1062,90,0.318000,130.0,449,0.551,20.000000,241.00000,0.165000,3.034056,0.866873,21.919505,1.857585,2.683179,9.267286,4.974200
5,eddie-house-1,Eddie House,Player,arizona state,405,19,0.495,0.425,0.187,0.798,124,73,4136.000000,2044,258,0.366000,250.0,502,0.523,10.800000,195.00000,0.113000,3.916828,0.183752,19.767892,2.495164,2.417795,4.854932,1.885880
6,eduardo-najera-1,Eduardo Najera,Player,oklahoma,215,89,0.460,0.430,0.354,0.669,126,80,3001.000000,1646,193,0.288000,85.0,910,0.495,13.100000,201.00000,0.183000,2.865711,1.186271,21.939354,2.572476,1.132956,12.129290,2.679107
7,lavor-postell-1,Lavor Postell,Player,st johns ny,205,58,0.470,0.430,0.383,0.789,129,77,2845.000000,1427,129,0.326000,92.0,716,0.526,13.800000,172.00000,0.115000,2.882250,0.815466,20.063269,1.813708,1.293497,10.066784,2.418278
8,hanno-mottola-1,Hanno Mottola,Bust,utah,139,50,0.533,0.500,0.415,0.774,120,83,2162.000000,1492,37,0.333000,71.0,551,0.580,14.900000,159.00000,0.207000,2.571693,0.925069,27.604070,0.684551,1.313599,10.194265,2.941721
9,chris-carrawell-1,Chris Carrawell,Bust,duke,309,114,0.514,0.486,0.415,0.667,136,78,2497.458432,1455,105,0.360000,62.0,608,0.545,15.209338,177.27896,0.181716,3.018494,1.357832,18.738180,1.505259,1.288549,8.322600,2.901975


In [416]:
df.to_csv("NCAA_data.csv",index = False, header = True)